In [ ]:
import torch
import torch.nn as nn
import numpy as np
import iisignature
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch.optim as optim

In [ ]:
def generate_ou_process(n_samples, n_steps, mu=0, theta=0.15, sigma=0.3):
    dt = 1.0
    X = np.zeros((n_samples, n_steps, 1), dtype=np.float32)
    X[:,0,0] = mu
    for t in range(1, n_steps):
        dX = theta * (mu - X[:, t-1, 0]) * dt + sigma * np.random.normal(size=n_samples) * np.sqrt(dt)
        X[:, t, 0] = X[:, t-1,0] + dX
    return X

In [ ]:
def generate_multidim_ou_process(n_samples, n_steps, mu=0, theta=0.15, sigma=0.3, n_features=1):
    dt = 1.0
    X = np.zeros((n_samples, n_steps, n_features), dtype=np.float32)
    X[:,0,:] = mu
    for t in range(1, n_steps):
        dX = theta * (mu - X[:, t-1, :]) * dt + sigma * np.random.normal(size=(n_samples, n_features)) * np.sqrt(dt)
        X[:, t, :] = X[:, t-1, :] + dX
    return X

In [ ]:
def AddTimeline(X):
    samples, steps, features = X.shape
    timeline = np.linspace(0, 1, steps)  # shape: (steps,)
    timeline = np.tile(timeline, (samples, 1))  # shape: (samples, steps)
    timeline = timeline[:, :, np.newaxis]  # shape: (samples, steps, 1)

    X_new = np.concatenate((timeline, X), axis=2)  # вставка timeline как первой фичи
    return X_new

In [ ]:
def full_signature(X, sig_level):
    '''
    X : (samples, steps, features) including time feature
    '''
    n_samples, n_steps, n_features = X.shape
    sig_length = iisignature.siglength(n_features, sig_level)

    Y = np.zeros((n_samples, n_steps, sig_length), dtype=np.float32)
    for i in range(n_samples):
        for j in range(n_steps):
            Y[i, j] = iisignature.sig(X[i, : j+1], sig_level)
    return Y

In [ ]:
def full_logsignature(X, sig_level):
    '''
    X : (samples, steps, features) including time feature
    '''
    n_samples, n_steps, n_features = X.shape
    s = iisignature.prepare(n_features, sig_level)
    sig_length = iisignature.logsiglength(n_features, sig_level)

    Y = np.zeros((n_samples, n_steps, sig_length), dtype=np.float32)
    for i in range(n_samples):
        for j in range(n_steps):
            Y[i, j] = iisignature.logsig(X[i, : j+1], s)
    return Y

In [ ]:
def RandomizedSignatureBatch(X_batch, k, activation='tanh', seed=None):
    '''
    Compute Randomized Signature with random matrixes A, b shared between samples
    X_batch : (samples, timesteps, features)
    
    activation func: 'linear', 'tanh', 'relu' or callable

    seed : random seed

    Returns Z_batch : (samples, steps, k)
    k-dimensional r-sig at each time step for each sample.
    '''
    X_batch = np.asarray(X_batch)
    samples, n, d = X_batch.shape
    rng = np.random.default_rng(seed)

    A = rng.standard_normal(size=(d, k, k))
    b = rng.standard_normal(size=(d, k))

    Z_batch = np.zeros((samples, n, k))
    Z_batch[:, 0, :] = rng.standard_normal(size=(samples, k))

    if activation == 'linear':
        def sigma(x): return x / np.sqrt(k)
    elif activation == 'tanh':
        sigma = np.tanh
    elif activation == 'relu':
        sigma = lambda x: np.maximum(x, 0)
    elif callable(activation):
        sigma = activation
    else:
        raise ValueError("Unsupported activation")

    dX_batch = np.diff(X_batch, axis=1)

    for j in range(1, n):
        z_prev = Z_batch[:, j-1, :]
        increment = np.zeros_like(z_prev)
        for i in range(d):
            projected = (z_prev @ A[i].T) + b[i]
            increment += sigma(projected) * dX_batch[:, j-1, i:i+1]
        Z_batch[:, j, :] = z_prev + increment

    return Z_batch

In [ ]:
def find_threshold(scores, true_labels):
    thresholds = np.unique(scores)  # > treshold => 1
    best_accuracy = 0
    best_f1 = 0
    best_accuracy_threshold = 0
    best_f1_threshold = 0

    for threshold in thresholds:
        predicted_labels = (scores > threshold).astype(int)

        accuracy = accuracy_score(true_labels, predicted_labels)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_accuracy_threshold = threshold

        f1 = f1_score(true_labels, predicted_labels)
        if f1 > best_f1:
            best_f1 = f1
            best_f1_threshold = threshold

    return best_accuracy_threshold, best_f1_threshold, best_accuracy, best_f1

In [ ]:
class RNNAutoencoder(nn.Module):
    '''
    RNN autoencoder with repeated input(last hidden state) to decoder in direct order(from 0 timestep to n-1)
    '''
    def __init__(self, input_dim, hidden_dim, latent_dim, rnn_type,
                 rnn_layers=1, activation=nn.ReLU, dropout=0.2):
        super().__init__()

        if rnn_type == 'gru':
            self.encoder_rnn = nn.GRU(input_dim, hidden_dim, num_layers=rnn_layers, batch_first=True)
            self.decoder_rnn = nn.GRU(hidden_dim, input_dim, num_layers=rnn_layers, batch_first=True)
        elif rnn_type == 'lstm':
            self.encoder_rnn = nn.LSTM(input_dim, hidden_dim, num_layers=rnn_layers, batch_first=True)
            self.decoder_rnn = nn.LSTM(hidden_dim, input_dim, num_layers=rnn_layers, batch_first=True)
        else:
            raise ValueError("Unsupported RNN type")

        self.fc_enc = nn.Linear(hidden_dim, latent_dim)
        self.fc_dec = nn.Linear(latent_dim, hidden_dim)

    def forward(self, x):
        batch_size, steps, _ = x.size()
        encoded_seq, hidden = self.encoder_rnn(x)

        if isinstance(hidden, tuple):  # LSTM
            hidden = hidden[0]

        hidden_last = hidden[-1]

        latent = self.fc_enc(hidden_last)

        decoder_input = self.fc_dec(latent).unsqueeze(1).repeat(1, steps, 1)
        decoded_seq, _ = self.decoder_rnn(decoder_input)

        return decoded_seq

In [ ]:
class DualMetricEarlyStopping:
    def __init__(self, patience : int, min_delta : float):
        '''
        patience: сколько эпох ждать перед остановкой
        min_delta: минимальное относительное улучшение (0.01 = 1%)
        "'''
        self.patience = patience
        self.min_delta = min_delta
        self.history = []

    def step(self, accuracy, roc_auc):
        self.history.append((accuracy, roc_auc))
        
        if len(self.history) <= self.patience:
            return False 

        recent = self.history[-self.patience-1:]
        base_acc, base_auc = recent[0]
        improved = False

        for acc, auc in recent[1:]:
            acc_gain = (acc - base_acc) / max(base_acc, 1e-6)
            auc_gain = (auc - base_auc) / max(base_auc, 1e-6)

            if acc_gain >= self.min_delta or auc_gain >= self.min_delta:
                improved = True
                break

        return not improved

def train_RNN(X_normal, X_test, labels, model_params, train_params, device=None):
    '''
    Train an RNN Autoencoder on normal data with validation on X_test
    '''
    num_epochs = train_params["num_epochs"]
    lr = train_params["lr"]
    batch_size = train_params["batch_size"]
    weight_decay = train_params["weight_decay"]
    patience = train_params["patience"]
    min_delta = train_params["min_delta"]

    best_acc = 0
    best_acc_epoch = -1
    best_roc_auc = 0
    best_roc_auc_epoch = -1
    best_metrics = {}
    
    if (train_params["standard_scaler"]) :
        samples, timesteps, features = X_normal.shape
        test_samples = X_test.shape[0]
        X_normal_reshaped = X_normal.reshape(-1, features)
        X_test_reshaped = X_test.reshape(-1, features)
    
        scaler = StandardScaler()
        X_normal_scaled = scaler.fit_transform(X_normal_reshaped)
        X_test_scaled = scaler.transform(X_test_reshaped)
    
        X_normal = X_normal_scaled.reshape(samples, timesteps, features)
        X_test = X_test_scaled.reshape(test_samples, timesteps, features)

    metrics = np.zeros((num_epochs, 2))

    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if isinstance(X_normal, np.ndarray):
        X_normal = torch.from_numpy(X_normal).float()
        
    dataset = TensorDataset(X_normal)
    loader = DataLoader(dataset, batch_size=train_params['batch_size'], shuffle=True)

    model = RNNAutoencoder(
        input_dim=X_normal.shape[-1],
        hidden_dim=model_params['hidden_dim'],
        latent_dim = model_params['latent_dim'],
        rnn_type=model_params['rnn_type'],
        rnn_layers = model_params['rnn_layers'],
        dropout=model_params['dropout']
    ).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=num_epochs//8, gamma=0.8)
    criterion = nn.MSELoss()

    early_stopper = DualMetricEarlyStopping(patience, min_delta)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for (x_batch,) in loader:
            x_batch = x_batch.to(device)

            optimizer.zero_grad()
            x_recon = model(x_batch)
            loss = criterion(x_recon, x_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * x_batch.size(0)

        epoch_loss /= len(loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.6f} | LR: {scheduler.get_last_lr()[0]:.6f}")
        (treshold_acc, treshold_f1, accuracy, best_f1), roc_auc = esteem_test_RNN(model, X_test, labels)
        print(f"Accuracy: {accuracy:.3f}, ROC-AUC: {roc_auc:.3f}")

        scheduler.step()

        if best_acc < accuracy:
                best_acc = accuracy
                best_acc_epoch = epoch+1
                
        if best_roc_auc < roc_auc:
            best_roc_auc = roc_auc
            best_roc_auc_epoch = epoch+1

        if early_stopper.step(accuracy, roc_auc):
                print(f"Early stopping at epoch {epoch+1}")
                break

    best_metrics = {
        "accuracy" : best_acc,
        "roc_auc" : best_roc_auc,
        "best_acc_epoch" : best_acc_epoch,
        "best_roc_auc_epoch" : best_roc_auc_epoch
        }

    return best_metrics

def esteem_test_RNN(RNN, RNN_test, labels_test):
    '''
    Test RNN model on RNN_test (samples, timesteps, features)
    Return: (best_accuracy_threshold, best_f1_threshold, best_accuracy, best_f1), roc_auc
    '''

    (samples, steps, sig_len) = RNN_test.shape
    esteems = np.zeros((samples))

    if not isinstance(RNN_test, torch.Tensor):
        RNN_test = torch.tensor(RNN_test, dtype=torch.float32).to(device)

    for i in range(samples):  
        RNN.eval()
        with torch.no_grad():
            x = RNN_test[i].reshape(1, steps, sig_len)
            reconstructed = RNN(x)
            loss = nn.MSELoss()(reconstructed, x)
            esteems[i] = loss.item()
    return find_threshold(esteems, labels_test), roc_auc_score(labels_test, esteems)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
train_samples = 50_000
test_samples = 2_000
anomaly_test_samples = int(test_samples * 0.5)
steps = 20

Одномерные процессы О-У

In [ ]:
np.random.seed(123)
train = generate_ou_process(train_samples, steps, 0, 0.15, 0.3)
test_1 = generate_ou_process(test_samples//2, steps, 0, 0.2, 0.4)
test_2 = generate_ou_process(test_samples//2, steps, 0, 0.15, 0.3)
test = np.concatenate([test_1, test_2], axis=0)
test_labels = np.zeros((test_samples)).astype(int)
test_labels[:test_samples//2] = 1

Десятимерные с аномалией в случайном измерении

In [ ]:
np.random.seed(123)
features = 10
train = generate_multidim_ou_process(train_samples, steps, 0, 0.15, 0.3, features)
test_1 = generate_multidim_ou_process(anomaly_test_samples, steps, 0, 0.15, 0.3, features)
for i in range(len(test_1)):
    test_1[i, :, i%features] = generate_ou_process(1, steps, 0, 0.3, 0.6).reshape(steps)
test_2 = generate_multidim_ou_process(test_samples - anomaly_test_samples, steps, 0, 0.15, 0.3, features)
test = np.concatenate([test_1, test_2], axis=0)
test_labels = np.zeros((test_samples)).astype(int)
test_labels[:anomaly_test_samples] = 1

Рандомизированная полная сигнатура

In [ ]:
r_sig_len = 500
train_r_sig = AddTimeline(train)
test_r_sig = AddTimeline(test)
train_r_sig = RandomizedSignatureBatch(train_r_sig, r_sig_len, activation="tanh")
test_r_sig = RandomizedSignatureBatch(test_r_sig, r_sig_len, activation="tanh")

Обычная полная сигнатура

In [ ]:
print(iisignature.siglength(2, 7))

In [ ]:
sig_level = 7
train_sig = AddTimeline(train)
test_sig = AddTimeline(test)
train_sig = full_signature(train, sig_level)
test_sig = full_signature(test, sig_level)

Полная лог-сигнатура

In [ ]:
print(iisignature.logsiglength(2, 10))

In [ ]:
logsig_level = 10
train_logsig = AddTimeline(train)
test_logsig = AddTimeline(test)
train_logsig = full_logsignature(train_logsig, logsig_level)
test_logsig = full_logsignature(test_logsig, logsig_level)

In [ ]:
RNN_params = {
    'hidden_dim' : 508,          # 2 * features
    'latent_dim' : 127,         # features / 2
    'rnn_type': 'lstm',
    'rnn_layers' : 1,
    'dropout' : 0.2
}

RNN_train_params = {
    'num_epochs': 100,
    'batch_size': 64,
    'lr': 0.001,
    'weight_decay' : 1e-4,
    "standard_scaler" : True,
    "patience" : 25,         # количество эпох которое даёт early stopper
    "min_delta" : 0.01       # минимальное увеличение метрик для early stopping
}

In [ ]:
metric = train_RNN(train_sig, test_sig, test_labels, RNN_params, RNN_train_params, device)
print(metric)